In [1]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import requests
import json
import datetime

# 1. Extract

Aqui, os dados são coletados da web (IMDb) usando o Selenium para navegar na página e BeautifulSoup para extrair as informações. Aqui acontece o seguinte:

- O Selenium é usado para navegar até a página de episódios e carregar todos os episódios.
- Ele localiza o botão "Tudo" e clica para carregar a lista completa.
- Os links de cada episódio são extraídos e armazenados na lista episodes.

In [2]:
def criar_banco_dados():
    conn = sqlite3.connect('naruto_episodes.db')
    cursor = conn.cursor()
    
    # Cria a tabela se não existir
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS episodios (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        numero_episodio INTEGER,
        titulo TEXT,
        data_lancamento DATE,
        votos INTEGER,
        avaliacao REAL,
        url_episodio TEXT,
        data_coleta DATETIME DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    conn.commit()
    return conn


In [3]:
#função para inserir dados no banco 
def inserir_episodio(conn, dados):
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO episodios (numero_episodio, titulo, data_lancamento, votos, avaliacao, url_episodio)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', dados)
    conn.commit()

In [4]:
#conecta ao banco de dados 
conn = criar_banco_dados()

In [5]:
#inicia o navegador 
driver = webdriver.Chrome()

#acessa a página do IMDb
url = 'https://www.imdb.com/title/tt0988824/episodes/'
driver.get(url)

#aguarda alguns segundos para garantir o carregamento 
time.sleep(3)

#clica no botão "Tudo" para carregar todos os episódios disponíveis 
try:
    button1 = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, 
        '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[2]/article[51]/div/span[2]')))
    
    driver.execute_script("arguments[0].scrollIntoView(true);", button1)

    time.sleep(3)

    button1.click()
    time.sleep(10)

    episodes = [] #cria uma lista para salvar os episódios 

    for i in range(1, 501): #quantidade de episódios disponíveis 

        try:
            episode = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[2]/article[{i}]/div/div/div[3]/div[1]/h4/div/a').get_attribute('href')
        except:
            try:
                episode = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[2]/article[{i}]/div/div/div[3]/div[2]/h4/div/a').get_attribute('href')
            except:
                print(f'Algo deu errado com o episódio: {i}')
            else:
                print(f'Episódio {i} encontrado: {episode}')
        else:
            print(f'Episódio {i} encontrado: {episode}')
        
        episodes.append(episode) #salva os episódios encontrados numa lista

except Exception as e:
    print(e)

finally:
    driver.close()


Episódio 1 encontrado: https://www.imdb.com/title/tt0990165/?ref_=ttep_ep_1
Episódio 2 encontrado: https://www.imdb.com/title/tt0990163/?ref_=ttep_ep_2
Episódio 3 encontrado: https://www.imdb.com/title/tt0990167/?ref_=ttep_ep_3
Episódio 4 encontrado: https://www.imdb.com/title/tt0990168/?ref_=ttep_ep_4
Episódio 5 encontrado: https://www.imdb.com/title/tt0990164/?ref_=ttep_ep_5
Episódio 6 encontrado: https://www.imdb.com/title/tt0990166/?ref_=ttep_ep_6
Episódio 7 encontrado: https://www.imdb.com/title/tt0996817/?ref_=ttep_ep_7
Episódio 8 encontrado: https://www.imdb.com/title/tt1191046/?ref_=ttep_ep_8
Episódio 9 encontrado: https://www.imdb.com/title/tt1202309/?ref_=ttep_ep_9
Episódio 10 encontrado: https://www.imdb.com/title/tt1202302/?ref_=ttep_ep_10
Episódio 11 encontrado: https://www.imdb.com/title/tt1202307/?ref_=ttep_ep_11
Episódio 12 encontrado: https://www.imdb.com/title/tt1202305/?ref_=ttep_ep_12
Episódio 13 encontrado: https://www.imdb.com/title/tt1202306/?ref_=ttep_ep_13
Epis

# 2. Transform 

Aqui, os dados brutos extraídos são processados e estruturados no formato adequado antes de serem carregados no banco de dados. Aqui acontece o seguinte:

+ Para cada episódio, o BeautifulSoup analisa o HTML e extrai os dados estruturados da página
+ O JSON é carregado para acessar os detalhes do episódio, como número, título, data de lançamento, votos e avaliação.
+ A data de lançamento é transformada para o formato datetime.date(ano, mês, dia), adequado para o banco de dados.

In [6]:
ep_count = 1

for episode in episodes:

    try:

        #simula o comportamento de um navegador adicionando um cabeçalho 'User-Agent'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        #faz a requisição usando o cabeçalho
        response = requests.get(episode, headers=headers)
        response.raise_for_status() #verifica se houve erro na requisição

        soup = BeautifulSoup(response.content, 'html.parser')  
        script = soup.find('script', id="__NEXT_DATA__", type="application/json")

        if not script:
            print(f"Não foi possível encontrar dados JSON para o episódio {ep_count}")
            continue

        json_data = script.string

        #converte a string JSON em um dicionário Python
        data = json.loads(json_data)

        #acessa as chaves dentro do dicionário
        content_data = data['props']['pageProps']['aboveTheFoldData']

        #extrai os dados
        num_episodio = content_data['series']['episodeNumber']['episodeNumber']
        titulo = content_data['titleText']['text']

        dia = content_data['releaseDate']['day'] 
        mes = content_data['releaseDate']['month'] 
        ano = content_data['releaseDate']['year']
        data_lancamento = datetime.date(ano, mes, dia)

        votos = content_data['ratingsSummary']['voteCount']
        avaliacao = content_data['ratingsSummary']['aggregateRating']

        #prepara os dados para a inserção
        dados_episodio = (num_episodio, titulo, data_lancamento, votos, avaliacao, episode)

        #insere no banco de dados
        inserir_episodio(conn, dados_episodio)

        print(f'Episódio {ep_count} processado e armazenado no banco de dados.')

    except Exception as e:
        print(f'Erro ao processar episódio {ep_count}: {str(e)}')

    ep_count += 1

#fecha a conexão com o banco de dados
conn.close()


Episódio 1 processado e armazenado no banco de dados.
Episódio 2 processado e armazenado no banco de dados.
Episódio 3 processado e armazenado no banco de dados.
Episódio 4 processado e armazenado no banco de dados.
Episódio 5 processado e armazenado no banco de dados.
Episódio 6 processado e armazenado no banco de dados.
Episódio 7 processado e armazenado no banco de dados.
Episódio 8 processado e armazenado no banco de dados.
Episódio 9 processado e armazenado no banco de dados.
Episódio 10 processado e armazenado no banco de dados.
Episódio 11 processado e armazenado no banco de dados.
Episódio 12 processado e armazenado no banco de dados.
Episódio 13 processado e armazenado no banco de dados.
Episódio 14 processado e armazenado no banco de dados.
Episódio 15 processado e armazenado no banco de dados.
Episódio 16 processado e armazenado no banco de dados.
Episódio 17 processado e armazenado no banco de dados.
Episódio 18 processado e armazenado no banco de dados.
Episódio 19 process

# 3. Load

Depois de extrair e transformar os dados, eles são armazenados no banco de dados SQLite. Aqui, acontece o seguinte:

+ Criamos uma conexão com o banco de dados SQLite (imdb_naruto.db).
+ Criamos a tabela episodes, se ainda não existir.
+ Inserimos cada episódio no banco de dados com um INSERT INTO.
+ Usamos commit() para garantir que os dados sejam realmente salvos.

In [7]:
import sqlite3

# Conectar ao banco
conn = sqlite3.connect('naruto_episodes.db')
cursor = conn.cursor()

# Exemplo de consulta
cursor.execute("SELECT * FROM episodios ORDER BY numero_episodio")
resultados = cursor.fetchall()

for linha in resultados:
    print(linha)

conn.close()

(1, 1, 'Kikyô', '2007-02-15', 1648, 8.2, 'https://www.imdb.com/title/tt0990165/?ref_=ttep_ep_1', '2025-03-25 19:35:21')
(2, 2, 'Akatsuki, shidô', '2007-02-15', 1217, 7.8, 'https://www.imdb.com/title/tt0990163/?ref_=ttep_ep_2', '2025-03-25 19:35:23')
(3, 3, 'Shûgyô no seika', '2007-02-22', 1082, 7.9, 'https://www.imdb.com/title/tt0990167/?ref_=ttep_ep_3', '2025-03-25 19:35:24')
(4, 4, 'Suna no jinchûriki', '2007-03-01', 1087, 8.2, 'https://www.imdb.com/title/tt0990168/?ref_=ttep_ep_4', '2025-03-25 19:35:26')
(5, 5, 'Kazekage to shite...!', '2007-03-15', 1030, 8.1, 'https://www.imdb.com/title/tt0990164/?ref_=ttep_ep_5', '2025-03-25 19:35:27')
(6, 6, 'Noruma kuriâ', '2007-03-29', 1012, 8.1, 'https://www.imdb.com/title/tt0990166/?ref_=ttep_ep_6', '2025-03-25 19:35:28')
(7, 7, 'Shissôre Kankurô', '2007-03-29', 958, 7.6, 'https://www.imdb.com/title/tt0996817/?ref_=ttep_ep_7', '2025-03-25 19:35:29')
(8, 8, 'Shutsugeki, Kakashi han', '2007-04-12', 925, 7.6, 'https://www.imdb.com/title/tt119104

In [8]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados
conn = sqlite3.connect('naruto_episodes.db')

# Ler os dados para um DataFrame do pandas
df = pd.read_sql_query("SELECT * FROM episodios", conn)

# Mostrar as primeiras linhas
print(df.head())

# Mostrar estatísticas básicas
print("\nEstatísticas descritivas:")
print(df.describe())

# Fechar a conexão
conn.close()

   id  numero_episodio                 titulo data_lancamento  votos  \
0   1                1                  Kikyô      2007-02-15   1648   
1   2                2        Akatsuki, shidô      2007-02-15   1217   
2   3                3        Shûgyô no seika      2007-02-22   1082   
3   4                4     Suna no jinchûriki      2007-03-01   1087   
4   5                5  Kazekage to shite...!      2007-03-15   1030   

   avaliacao                                       url_episodio  \
0        8.2  https://www.imdb.com/title/tt0990165/?ref_=tte...   
1        7.8  https://www.imdb.com/title/tt0990163/?ref_=tte...   
2        7.9  https://www.imdb.com/title/tt0990167/?ref_=tte...   
3        8.2  https://www.imdb.com/title/tt0990168/?ref_=tte...   
4        8.1  https://www.imdb.com/title/tt0990164/?ref_=tte...   

           data_coleta  
0  2025-03-25 19:35:21  
1  2025-03-25 19:35:23  
2  2025-03-25 19:35:24  
3  2025-03-25 19:35:26  
4  2025-03-25 19:35:27  

Estatísticas 